In [ ]:
import mapsims
import numpy as np
import healpy as hp
import pysm3.units as u
from pathlib import Path
import toml
%matplotlib inline
import matplotlib.pyplot as plt

# Run map-based simulations on-the-fly starting from configuration files

Once the Map-Based-Simulations group publishes a simulation release, it will provide configuration files which setup `mapsims` with all the correct parameters,
see for example the 2 configuration toml (like `.ini` files <https://github.com/toml-lang/toml>) files: 

In [ ]:
!git clone --depth 1 https://github.com/simonsobs/map_based_simulations.git

In [ ]:
!ls map_based_simulations/202006_noise/*.toml

Multiple configuration files can be fed to `mapsims`, the `common.toml` generally contains general information about the output path, naming, units, simulation seed and splits.

The `num` parameter is very important because it is passed thorugh to all the other simulation classes. So if you change that to 1, all the components a that support a seed, for example also CMB, creates a new realization which is reproducible.

In [ ]:
!cat map_based_simulations/202006_noise/common.toml

The TOML files for each component are very general, they get a `class` attribute and then every other attribute is just passed to the class constructor, therefore it can execute any class as long as it has a standard `simulate` method with the expected signature.

I also have a command line tool that can directly produce simulations from configuration files, `mapsims_run`, which I use to produce the map-based simulation releases. But we can also use those configuration files as a starting point for custom runs using the Python classes.

In [ ]:
!cat map_based_simulations/202006_noise/noise.toml

The simplest way is to just copy-paste this configuration directly into a class constructor or load it with TOML, which creates a dictionary from a `.toml` file.

In [ ]:
noise_config = toml.load("map_based_simulations/202006_noise/noise.toml")["other_components"]["noise"]

In [ ]:
noise_config

In [ ]:
noise_config.pop("class")

In [ ]:
noise_sim = mapsims.SONoiseSimulator(nside=128, **noise_config)

# Use the map simulator class MapSim

We load the configuration files and then we override whatever configuration option we need,
or we could create another TOML file and load it after `noise.toml`.
`num` is the realization number, i.e. it fixes the seed for all different channels,
the set of maps released were generated with `num=0`, we can set it to 1 to get a different set.

We always work 1 tube at a time and get 2 maps with a cross-correlated component given by the atmosphere:

In [ ]:
sim = mapsims.from_config(["map_based_simulations/202006_noise/common.toml",
                           "map_based_simulations/202006_noise/noise.toml"],
                           override={"channels":"tube:ST3", "output_folder":".", "num":1})

In [ ]:
type(sim)

Finally call execute to generate and return the maps.

NSIDE is automatically set based on the channel, you can also override that by providing `nside` key to `override`.

In [ ]:
noise = sim.other_components["noise"]

In [ ]:
maps_1 = sim.execute()

In [ ]:
maps_1.keys()

In [ ]:
maps_1["ST3_LF1"].shape

To get another realization you can either create another `sim` object with a different `num`
or directly override it on a already existing object:

In [ ]:
sim.num = 2 # this is only used for the output filename
sim.other_components["noise"].seed = 2 # this is the actual realization number
maps_2 = sim.execute()

To change tube without creating another `sim` object, override `sim.channels`:

In [ ]:
sim.channels

In [ ]:
sim.channels = mapsims.parse_channels("tube:ST0")

In [ ]:
sim.execute().keys()

## Write output maps

In [ ]:
# save fits files instead
sim.execute(write_outputs=True)

In [ ]:
%ls *.fits

## Plot the output maps

In [ ]:
import healpy as hp

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(maps_1["ST3_LF1"][1], min=-50, max=50)

In [ ]:
hp.mollview(hp.ma(maps_2["ST3_LF1"][1]), min=-50, max=50)

# Generate multiple splits

Override `nsplits` to generate multiple splits, the output maps will be a 3 dimensional array where for each split we have the 3 IQU components.

In [ ]:
sim = mapsims.from_config(["map_based_simulations/202006_noise/common.toml",
                           "map_based_simulations/202006_noise/noise.toml"],
                          override={"channels":"tube:ST3", "output_folder":".", "num":1, "nsplits":4})

In [ ]:
maps = sim.execute()

In [ ]:
maps["ST3_LF1"].shape

# Generate noise and CMB

We can also check other map-based simulations releases and combine different configuration files to produce on-the-fly maps which have multiple components, for example noise and CMB.

The `201911_lensed_cmb` includes 100 lensed CMB realizations, we can grab the `cmb.toml` file used to generate them, customize it with the correct path for NERSC, and feed it into `mapsims.from_config`.

In [ ]:
!ls map_based_simulations/201911_lensed_cmb

In [ ]:
!cat map_based_simulations/201911_lensed_cmb/cmb.toml

In [ ]:
%%file cmb.toml

tag = "cmb"

[ other_components ]

    [ other_components.cmb ]
    class  =  "mapsims.SOStandalonePrecomputedCMB"
    lensed = true
    # num is then overridden by command line option to mapsims_run
    num = 0
    aberrated = false
    has_polarization = true
    cmb_set = 0
    # At NERSC use:
    cmb_dir = "/global/project/projectdirs/sobs/v4_sims/mbs/cmb"
    # On Popeye use:
    # cmb_dir = "/simons/scratch/zonca/simonsobs/lensed_cmb"
    input_units = "uK_CMB"

In [ ]:
sim_noise_cmb = mapsims.from_config(["map_based_simulations/202006_noise/common.toml",
                           "map_based_simulations/202006_noise/noise.toml",
                           "cmb.toml"],
                          override={"channels":"tube:ST3", "output_folder":".", "num":1, "nsplits":1})

In [ ]:
sim_noise_cmb.other_components

In [ ]:
maps_noise_cmb = sim_noise_cmb.execute()

In [ ]:
hp.mollview(hp.ma(maps_noise_cmb["ST3_LF1"][0]), title="Noise + CMB", unit="$\mu K_{CMB}$", min=-100, max=100)

# Add foreground components

We can also look at the older release `201906_highres_foregrounds_extragalactic_tophat` we saw in notebook 1, but now, instead of reading maps from disk, we generate it on the fly, which is convenient if we want a different $N_{side}$.
Also, depending on the specific case, it might be quicker to generate components on-the-fly instead of loading multiple maps from disk./ 

In [ ]:
!ls map_based_simulations/201906_highres_foregrounds_extragalactic_tophat/

In [ ]:
!cat map_based_simulations/201906_highres_foregrounds_extragalactic_tophat/synchrotron_512.toml

In [ ]:
sim_noise_cmb_sync = mapsims.from_config(["map_based_simulations/202006_noise/common.toml",
                           "map_based_simulations/202006_noise/noise.toml",
                           "cmb.toml",
                           "map_based_simulations/201906_highres_foregrounds_extragalactic_tophat/synchrotron_512.toml"
                          ],
                          override={"channels":"tube:ST3", "output_folder":".", "num":1, "nsplits":1})

In [ ]:
maps_noise_cmb_sync = sim_noise_cmb_sync.execute()

In [ ]:
hp.mollview(hp.ma(maps_noise_cmb_sync["ST3_LF1"][0]), title="Noise + CMB + Synchrotron", unit="$\mu K_{CMB}$", min=-100, max=100)